## Building a Brandon Sanderson GPT

Notebook made with [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [55]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/calderrussell/BSLLM/main/data_new.txt

--2025-07-03 18:43:44--  https://raw.githubusercontent.com/calderrussell/BSLLM/main/data_new.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6261732 (6.0M) [application/octet-stream]
Saving to: ‘data_new.txt.2’

data_new.txt.2      100%[===================>]   5.97M  --.-KB/s    in 0.06s   

2025-07-03 18:43:44 (104 MB/s) - ‘data_new.txt.2’ saved [6261732/6261732]



In [56]:
# read it in to inspect it
with open('data_new.txt', 'r', encoding='mac_roman') as f:
    text = f.read()

In [57]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  6261732


In [58]:
# let's look at the first 1000 characters
print(text[:5000])



FOR JOSHUA BILMES 
Who is never afraid to tell me what is wrong with a book, then ﬁght for that same book no matter who else gives up on it. 

ACKNOWLEDGMENTS 
I ﬁrst pitched the idea of later-era Mistborn novels to my editor back in 2006, I believe. It had long been my plan for Scadrial, the planet these books take place upon. I wanted to move away from the idea of fantasy worlds as static places, where millennia would pass and technology would never change. The plan then was for a second epic trilogy set in an urban era, and a third trilogy set in a futuristic era— with Allomancy, Feruchemy, and Hemalurgy being the common threads that tied them together. 
This book isn’t part of that second trilogy. It’s a side deviation, something exciting that grew quite unexpectedly out of my planning for where the world would go. The point of telling you all of this, however, is to explain that it would be impossible to list all of the people who have helped me along the years. Instead, the bes

In [59]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
chars = chars[:-15]
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$&'()*,-./0123456789:;=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]_abcdefghijklmnopqrstuvwxyz
87


In [60]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
filtered_text = ''.join([c for c in text if c in stoi])
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[68, 69, 69, 1, 80, 68, 65, 78, 65]
hii there


In [61]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(filtered_text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([6035142]) torch.int64
tensor([ 0,  0, 36, 45, 48,  1, 40, 45, 49, 38, 51, 31,  1, 32, 39, 42, 43, 35,
        49,  1,  0, 53, 68, 75,  1, 69, 79,  1, 74, 65, 82, 65, 78,  1, 61, 66,
        78, 61, 69, 64,  1, 80, 75,  1, 80, 65, 72, 72,  1, 73, 65,  1, 83, 68,
        61, 80,  1, 69, 79,  1, 83, 78, 75, 74, 67,  1, 83, 69, 80, 68,  1, 61,
         1, 62, 75, 75, 71, 11,  1, 80, 68, 65, 74,  1, 67, 68, 80,  1, 66, 75,
        78,  1, 80, 68, 61, 80,  1, 79, 61, 73, 65,  1, 62, 75, 75, 71,  1, 74,
        75,  1, 73, 61, 80, 80, 65, 78,  1, 83, 68, 75,  1, 65, 72, 79, 65,  1,
        67, 69, 82, 65, 79,  1, 81, 76,  1, 75, 74,  1, 69, 80, 13,  1,  0,  0,
        31, 33, 41, 44, 45, 53, 42, 35, 34, 37, 43, 35, 44, 50, 49,  1,  0, 39,
         1, 78, 79, 80,  1, 76, 69, 80, 63, 68, 65, 64,  1, 80, 68, 65,  1, 69,
        64, 65, 61,  1, 75, 66,  1, 72, 61, 80, 65, 78, 12, 65, 78, 61,  1, 43,
        69, 79, 80, 62, 75, 78, 74,  1, 74, 75, 82, 65, 72, 79,  1, 80, 75,  1,
      

In [62]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [63]:
block_size = 8
train_data[:block_size+1]

tensor([ 0,  0, 36, 45, 48,  1, 40, 45, 49])

In [64]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([0]) the target: 0
when input is tensor([0, 0]) the target: 36
when input is tensor([ 0,  0, 36]) the target: 45
when input is tensor([ 0,  0, 36, 45]) the target: 48
when input is tensor([ 0,  0, 36, 45, 48]) the target: 1
when input is tensor([ 0,  0, 36, 45, 48,  1]) the target: 40
when input is tensor([ 0,  0, 36, 45, 48,  1, 40]) the target: 45
when input is tensor([ 0,  0, 36, 45, 48,  1, 40, 45]) the target: 49


In [65]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[83, 75, 78, 78, 69, 65, 79,  1],
        [80, 68, 65, 64, 65, 66, 65, 61],
        [69, 79, 65, 64,  1, 61, 74,  1],
        [65, 13,  1, 31, 64, 61, 73,  1]])
targets:
torch.Size([4, 8])
tensor([[75, 78, 78, 69, 65, 79,  1, 73],
        [68, 65, 64, 65, 66, 65, 61, 80],
        [79, 65, 64,  1, 61, 74,  1, 65],
        [13,  1, 31, 64, 61, 73,  1, 38]])
----
when input is [83] the target: 75
when input is [83, 75] the target: 78
when input is [83, 75, 78] the target: 78
when input is [83, 75, 78, 78] the target: 69
when input is [83, 75, 78, 78, 69] the target: 65
when input is [83, 75, 78, 78, 69, 65] the target: 79
when input is [83, 75, 78, 78, 69, 65, 79] the target: 1
when input is [83, 75, 78, 78, 69, 65, 79, 1] the target: 73
when input is [80] the target: 68
when input is [80, 68] the target: 65
when input is [80, 68, 65] the target: 64
when input is [80, 68, 65, 64] the target: 65
when input is [80, 68, 65, 64, 65] the target: 66
when input

In [66]:
print(xb) # our input to the transformer

tensor([[83, 75, 78, 78, 69, 65, 79,  1],
        [80, 68, 65, 64, 65, 66, 65, 61],
        [69, 79, 65, 64,  1, 61, 74,  1],
        [65, 13,  1, 31, 64, 61, 73,  1]])


In [67]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 87])
tensor(5.2653, grad_fn=<NllLossBackward0>)

0(=2UQul,1'O>k[c/RpmlFTjfW4$-RWr"\ kL[4T_X?-UgAIP1;;e2q*d, CVNj.XBF*(=T?;C?eG*(0iIt
\38LeGpSDhvOOK0b


In [68]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [69]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.808799743652344


In [70]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


bZgqYLgTZ?qT?jn'$$cJo7rSqkX2g]'7XPrQe1/*1:&VhFQZa1T>=Xn9ww[4;(=/RS$?-F"POD3A42'3j)=KfWXZ$(QYMOB6ZM5iiH4O-R-#vcL(BLkL>O
ax1a8uYUK/)i-#R-jun
(=q*VSTru?ffkqF(LAsZ
?/I8Kg7,A47"/F26_uF,1!J-,&!0ryhZe=(Y==K?J-Ak550G5iq19fto:0:@s#
lCpEJCll) "ne6bUN1/=v5BeV#>Pp$mD4\a vQzy-#msFx7&4to!M\BkX;&Ci.!\o7gGeacoyh_(dqscm@*kaNqY,YH#mmM/I2bI1xKWX)"IDjG9!Y,&=L6Z?MeEe(nkAHTRQV!2Ux?R .WNBk_D/exV8LBVvq*L*Oy7Q:NGe&*uI]xO:I&L=D7g!XK 3uc9pxr-.?@PS9Bwc"Vxn1nz/zBxy U*Nq*sb0nk[8 4Vz7x2idY24G4Tr:Ur*9Q6(h>2F*9$_M]4[/R;:;E=>dBx


## The mathematical trick in self-attention

In [71]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [72]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [73]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [74]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [75]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [76]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [77]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [78]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [79]:
k.var()

tensor(1.0449)

In [80]:
q.var()

tensor(1.0700)

In [81]:
wei.var()

tensor(1.0918)

In [82]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [83]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [84]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [85]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [86]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [87]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [89]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
#dataBSLLM.txt
with open('data_new.txt', 'r', encoding='mac_roman') as f:
  #CP1252
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
chars = chars[:-15]
#chars = chars[:-11]
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
filtered_text = ''.join([c for c in text if c in stoi])
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(filtered_text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.212567 M parameters
step 0: train loss 4.6202, val loss 4.6268
step 100: train loss 2.6356, val loss 2.5813
step 200: train loss 2.5127, val loss 2.4671
step 300: train loss 2.4260, val loss 2.3952
step 400: train loss 2.3506, val loss 2.3211
step 500: train loss 2.2942, val loss 2.2511
step 600: train loss 2.2481, val loss 2.2053
step 700: train loss 2.1969, val loss 2.1469
step 800: train loss 2.1547, val loss 2.1061
step 900: train loss 2.0990, val loss 2.0682
step 1000: train loss 2.0800, val loss 2.0290
step 1100: train loss 2.0520, val loss 2.0049
step 1200: train loss 2.0343, val loss 1.9689
step 1300: train loss 2.0119, val loss 1.9352
step 1400: train loss 1.9697, val loss 1.9236
step 1500: train loss 1.9580, val loss 1.9004
step 1600: train loss 1.9427, val loss 1.8863
step 1700: train loss 1.9312, val loss 1.8577
step 1800: train loss 1.9035, val loss 1.8553
step 1900: train loss 1.8876, val loss 1.8245
step 2000: train loss 1.8809, val loss 1.8182
step 2100: train loss 1.